# Embeddings Neuronales


**TODO**

### Importanción de librería requeridas

In [24]:
import os

import warnings
import pickle
from collections import Counter

import numpy as np
import pandas as pd

import gensim.corpora as corpora
from gensim.models import FastText

from clustering_utils import vectorize, mbkmeans_clusters

warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline


### Definición de variables globales

In [2]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_fasttext.csv'
FILENAME_PICKLE = "docs/tmpreddit.pickle"
n_clusters = 70

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [3]:
with open(FILENAME_PICKLE, 'rb') as f:
    df = pickle.load(f)


### Vocabulario

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo FastText

In [5]:
model = FastText(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=4)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)

(18382040, 18695800)

In [6]:
# algunas predicciones
#model.get_nearest_neighbors('peron')

In [7]:
#model.get_nearest_neighbors('cristina')

In [8]:
#model.get_nearest_neighbors('nestor')

In [9]:
#model.get_nearest_neighbors('neoliberal')

In [10]:
#model.get_nearest_neighbors('malvinas')

In [11]:
#model.get_analogies('cristina', 'ladrona', 'alberto')

In [12]:
#model.wv.most_similar('mapuche')

### Generación de vectores desde documentos

In [13]:

vectorized_docs = vectorize(processed_corpus, model=model)
len(vectorized_docs), len(vectorized_docs[0])



(27791, 100)

### Generación de clusters

In [14]:
clustering, cluster_labels = mbkmeans_clusters(
    X=vectorized_docs,
    k=n_clusters,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": df["body"].values,
    "tokens": [" ".join(text) for text in processed_corpus],
    "cluster": cluster_labels
})



For n_clusters = 70
Silhouette coefficient: 0.01
Inertia:1647867.2361781183
Silhouette values:
    Cluster 11: Size:149 | Avg:0.20 | Min:0.06 | Max: 0.34
    Cluster 25: Size:148 | Avg:0.14 | Min:-0.04 | Max: 0.32
    Cluster 56: Size:163 | Avg:0.11 | Min:-0.05 | Max: 0.31
    Cluster 9: Size:62 | Avg:0.11 | Min:-0.03 | Max: 0.31
    Cluster 50: Size:326 | Avg:0.10 | Min:-0.02 | Max: 0.25
    Cluster 44: Size:239 | Avg:0.10 | Min:-0.05 | Max: 0.25
    Cluster 47: Size:214 | Avg:0.10 | Min:-0.03 | Max: 0.30
    Cluster 41: Size:56 | Avg:0.09 | Min:0.00 | Max: 0.24
    Cluster 59: Size:153 | Avg:0.09 | Min:-0.05 | Max: 0.27
    Cluster 3: Size:1058 | Avg:0.08 | Min:-0.03 | Max: 0.23
    Cluster 14: Size:195 | Avg:0.08 | Min:-0.05 | Max: 0.27
    Cluster 40: Size:26 | Avg:0.08 | Min:-0.05 | Max: 0.27
    Cluster 42: Size:168 | Avg:0.07 | Min:-0.05 | Max: 0.27
    Cluster 30: Size:364 | Avg:0.07 | Min:-0.06 | Max: 0.25
    Cluster 49: Size:85 | Avg:0.07 | Min:-0.04 | Max: 0.26
    Cluster 

### *Top terms* por cluster (basado en los centroides de los clusters)

In [15]:
print("Most representative terms per cluster (based on centroids):")
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")



Most representative terms per cluster (based on centroids):
Cluster 0: dar odar gaydar rodar sudar leudar amoldar apodar tildar anidar 
Cluster 1: pagar pagarar \*pagar pagartar apagar pagarl pagarlo prepagar divagar pagarme 
Cluster 2: plata planck plan platea platita platón platynun platar plazita plaz 
Cluster 3: él inocentemente intermitente eficientemente obligadamente inocente coeficiente intento ineficiente corruptamente 
Cluster 4: chinardo chevy china chevrolé che chispa chispo chi chistoso chifla 
Cluster 5: \+agente eficientemente exigente demente conscientemente coeficiente oyente agente ineficiente gente 
Cluster 6: hijaeputa hijo hijo_puta hijota hijos hijodeputo putamadre hij laputamadre puta 
Cluster 7: terrateniente coeficiente terriblemente eficiente enterrado fehaciente deficiente proveniente estupefaciente creciente 
Cluster 8: jajajar jajajajar jajaja jajajajjar jajajaja jajajajajar jajajaajaja jajajejox jajajajajjajar jajajajaja 
Cluster 9: tema tema" temas temaso

### *Top terms* por cluster (basado en las palabras más frecuentes)

In [16]:
for i in range(n_clusters):
    tokens_per_cluster = ""
    most_frequent = Counter(" ".join(df_clusters.query(f"cluster == {i}")["tokens"]).split()).most_common(5)
    for t in most_frequent:
        tokens_per_cluster += f"{t[0]}({str(t[1])}) "
    print(f"Cluster {i}: {tokens_per_cluster}")


Cluster 0: dar(243) él(91) quedar(21) vuelta(15) querer(15) 
Cluster 1: pagar(235) impuesto(73) cobrar(37) pago(26) sueldo(23) 
Cluster 2: plan(88) plata(75) plato(15) social(13) querer(13) 
Cluster 3: él(1199) querer(76) hacer(73) ver(56) dejar(54) 
Cluster 4: che(92) china(31) chino(29) onda(25) mina(15) 
Cluster 5: gente(264) persona(59) pasar(53) entender(35) pensar(34) 
Cluster 6: hijo(147) puta(86) puto(34) madre(27) padre(24) 
Cluster 7: tener(96) llegar(54) tenés(50) tirar(46) punto(45) 
Cluster 8: jaja(116) jajaja(85) jajajar(66) jajajaja(52) jajaj(40) 
Cluster 9: tema(63) aborto(3) gente(3) post(3) tocar(2) 
Cluster 10: problema(166) resolver(16) peronismo(8) volver(8) solucionar(7) 
Cluster 11: the(164) of(49) you(39) is(39) and(38) 
Cluster 12: lado(44) pelado(25) quedar(19) pedo(19) pasar(17) 
Cluster 13: precio(186) comprar(93) producto(23) compra(18) barato(18) 
Cluster 14: tipo(195) persona(9) tirar(7) gente(6) gustar(6) 
Cluster 15: vida(253) vivir(67) gente(13) él(12)

### Recupere los documentos más representativos (basados en los centroides de los clústeres) para un cluster en particular

In [17]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    print( df["body"].values[d])
    print("-------------")

También te podés enterar que darle un danonino y una barrita de cereales a un nene todos los dias no es para nada saludable
-------------
Si me traen a twice o mamamoo  esta todo bien. Total... Para lo que nos roban aunque sea que nos den un recital.. Ya no quiero a León gieco o fito Páez. Ni onda tienen ya.
-------------
Que salame jajajajaja. No pasa por el lado del miedo sino por el lado de no darle cabida; contacto visual es perpetuar la interacción entre los dos
-------------
Tal cual. Yo les doy una machinegun y un exo-esqueleto hidráulico con vision nocturna a cada uno.
-------------
Queda en nosotros, don Falla, buscar a quien darle con ese romanticismo guardado que tenemos adentro y que no le joda jajajaja
-------------
Boludo me dejaste regulando, recién me doy cuenta que era el disfraz de danielsan en karate kid jajaja
-------------
¿Mr. poronga por Detroit? Está a un doble sentido de distancia que le den cédula venezolana.
-------------
Yo también me quiero comprar un Passa

In [18]:

#print(len(vectorized_docs))
#print(vectorized_docs[0])

test_v = vectorize([['defender', 'peso', 'siente', 'corazón', 'compro', 'pesos', 'tasa', 'fijo', 'año']], model=model)
prediction = clustering.predict(test_v)
print(prediction)

[39]


In [19]:
reddit = pd.read_csv(TEXT_FILE_READ)

def get_cluster(row):
    test_v = vectorize([row], model=model)
    return clustering.predict(test_v)

reddit['cluster'] = reddit.apply(lambda row: get_cluster(row['lemma_tokens']) , axis = 1) 


In [20]:
# Show
reddit.head(10)

,score,id,flair,comms_num,body,comment_parent_id,is_replay,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,lemma_tokens,body_preprocessing,cluster
0,1,hfw14mt,Discusion🧐,1,todo para decir que tapaste el baño. tira un b...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tapastir', 'baño', 'tirar', 'balde', 'aguo']",tapastir baño tirar balde aguo,[24]
1,1,hfw41eh,Discusion🧐,0,"sopapa primero master, si hay tapón te vas a t...",hfw14mt,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapa', 'master', 'tapón', 'va', 'teñir', '...",sopapa master tapón va teñir medio,[24]
2,1,hfw1ao2,Discusion🧐,0,"Usas la sopapa, o tiras agua caliente con un b...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sopapo', 'tira', 'agua', 'caliente', 'balde']",sopapo tira agua caliente balde,[24]
3,1,hfw3jof,Discusion🧐,2,Lo que he probado que siempre me dio resultado...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['probado', 'resultado', 'sellar', 'boca', 'in...",probado resultado sellar boca inodoro tirar ca...,[24]
4,1,hfw6v4i,Discusion🧐,0,Estas cobrando por dar mantenimiento y no sabe...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['cobrar', 'mantenimiento', 'carajo', 'kjjjjjj...",cobrar mantenimiento carajo kjjjjjjjjj vivirio...,[24]
5,1,hfw26iv,Discusion🧐,0,"Si tenes algo con punta, metelo y hace un poco...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tén', 'punto', 'metelo', 'fuerza', 'romper',...",tén punto metelo fuerza romper tapo baño tirar...,[24]
6,1,hfw2gof,Discusion🧐,1,"Con una manguera para regar el jardín, si tene...",q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['regar', 'jardín', 'tén', 'pod', 'probar']",regar jardín tén pod probar,[24]
7,1,hfw5s13,Discusion🧐,0,"despues regas el jardin y se lava sola, solo q...",hfw2gof,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['rega', 'jardin', 'lava', 'tenés', 'lavarte',...",rega jardin lava tenés lavarte mano pulgar chorro,[24]
8,1,hfw3air,Discusion🧐,0,La respuesta real es que se venden unos caños ...,q44kw3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['respuesta', 'real', 'vender', 'caño', 'alamb...",respuesta real vender caño alambrado decir cañ...,[24]
9,7,hfvxa6w,Discusion🧐,3,Mi alfajor favorito es el Havana,q443eo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['alfajor', 'favorito', 'haván']",alfajor favorito haván,[24]


In [21]:
reddit.to_csv(TEXT_SAVE_FILE, index=False)

In [27]:
cluster_path = 'docs/test/fasttext_comments_per_cluster/'

os.makedirs(test_dir_path,exist_ok=True)

for i in range(n_clusters):
    reddit[(reddit["cluster"] == i)][['flair', 'body']].to_csv(test_dir_path + str(i) + '.csv')
